In [22]:
import pandas as pd
import ast
import pyGM as gm
import numpy as np
import math

In [23]:
#upload card db
cards = pd.read_csv('data/cards.csv')

In [24]:
# add converted mana cost as well as make all values not strings
cards['CMC'] = cards['manaCost'].apply(lambda x: ''.join(''.join(str(x).split('{')).split('}')))
for col in cards:
    try:
        cards[col] = cards[col].apply(lambda x: ast.literal_eval(x))
    except:
        pass
for index, row in cards.iterrows():
    X = row['CMC']
    list(X)
    x_ = 0
    for x in X:
        try:
            x_+=int(x)
        except:
            x_+=1
    row['CMC']=x_

In [25]:
# only keep modern legal cards and drop non useful cols
legal = []
for index, row in cards.iterrows():
    if 'modern' in row['legalities']:
        if row['legalities']['modern']=='Legal':
            legal.append(True)
        else:
            legal.append(False)
    else:
        legal.append(False)
cards = cards[legal]
cards.drop(columns=['legalities','Unnamed: 0','leadershipSkills','power','toughness','loyalty','uuid'],inplace=True)

In [26]:
cards

,colors,manaCost,name,subtypes,supertypes,text,type,types,hasNoDeckLimit,CMC
8,[R],{2}{R},Abandon Reason,[],[],"[discard, discard, cast, madness]",Instant,[Instant],NaN,3
10,[],{3},Abandoned Sarcophagus,[],[],"[cast, cycling, cycling, exile]",Artifact,[Artifact],NaN,3
11,[B],{3}{B},Abattoir Ghoul,[Zombie],[],[creature],Creature — Zombie,[Creature],NaN,4
13,[W],{3}{W},Abbey Griffin,[Griffin],[],[vigilance],Creature — Griffin,[Creature],NaN,4
15,[R],{1}{R},Abbot of Keral Keep,"[Human, Monk]",[],"[cast, creature, exile, play]",Creature — Human Monk,[Creature],NaN,2
...,...,...,...,...,...,...,...,...,...,...
21032,[B],{B},Zulaport Enforcer,"[Human, Warrior]",[],[counter],Creature — Human Warrior,[Creature],NaN,1
21034,"[B, U, W]",{1}{W}{U}{B},Zur the Enchanter,"[Human, Wizard]",[Legendary],"[search, enchantment, shuffle]",Legendary Creature — Human Wizard,[Creature],NaN,4
21035,[U],{3}{U},Zur's Weirding,[],[],"[play, reveal]",Enchantment,[Enchantment],NaN,4
21039,[R],{R},Zurgo Bellstriker,"[Orc, Warrior]",[Legendary],"[cast, dash]",Legendary Creature — Orc Warrior,[Creature],NaN,1


In [27]:
#upload decklists
decks = dict()
decks['modern2018'] = pd.read_csv('data/deckListmodernMetaDecks2018.csv')
decks['modern2019'] = pd.read_csv('data/deckListmodernMetaDecks2019.csv')
decks['modern2020'] = pd.read_csv('data/deckListmodernMetaDecks2020.csv')


In [28]:
#select only one archetype from decklists
decklists = dict()
decklists['modern2020'] = decks['modern2020'][decks['modern2020']['1']=='Bant Control']['3']
decklists['modern2019'] = decks['modern2019'][decks['modern2019']['1']=='Bant Control']['3']
decklists['modern2018'] = decks['modern2018'][decks['modern2018']['1']=='Bant Control']['3']
for d in decklists:
    print(decklists[d],d)
    
decklists['modern2020'] = decklists['modern2020'].to_dict()[28]
decklists['modern2020'] = ast.literal_eval(decklists['modern2020'])
decklists['modern2019'] = decklists['modern2019'].to_dict()[47]
decklists['modern2019'] = ast.literal_eval(decklists['modern2019'])


28    [{'LANDS': ['2 Breeding Pool', '3 Field of Rui...
Name: 3, dtype: object modern2020
47    [{'LANDS': ['1 Breeding Pool', '2 Celestial Co...
Name: 3, dtype: object modern2019
Series([], Name: 3, dtype: object) modern2018


In [29]:
x = len(decklists['modern2018'])
y = len(decklists['modern2019'])
z = len(decklists['modern2020'])

In [30]:
#Generate dataframes with card frequencies for all years for main and side
# as well as correlation matrixes for cards
simMain = dict()
simSide = dict()
for D in decklists:
    for i,deck in enumerate(decklists[D]):
        for ty in deck:
            if ty=='SIDEBOARD':
                sim=simSide
            else:
                sim=simMain
            for card in deck[ty]:
                card = card.split(' ')
                freq = card[0]
                name = ' '.join(card[1:])
                if name not in sim:
                    sim[name] = list()
                    if D=='modern2020':
                        i_ = i+x+y
                    if D=='modern2019':
                        i_ = i+x
                    if D=='modern2018':
                        i_ = i
                    sim[name].append([i_,int(freq)])
                else:
                    isIn = False
                    for i_,freq_ in sim[name]:
                        if i_==i:
                            isIn=True
                            freq_+=int(freq)
                    if isIn==False:
                        sim[name].append([i,int(freq)])
                        
                        
sim=simSide
columns=list(sim.keys())
d = pd.DataFrame(columns=columns)
for i in range(x+y+z):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1
corrSide=corr
dSide = d

sim=simMain
columns=list(sim.keys())
d = pd.DataFrame(columns=columns)
for i in range(x+y+z):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1

corrMain = corr
dMain = d

In [31]:
#select multiple potential cards and impute those values
#https://towardsdatascience.com/preprocessing-regression-imputation-of-missing-continuous-values-f612179bafb4
import pandas as pd
import numpy as np
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
import random

x_ = len(dMain)
dMain.loc[x_] = None
for i in range(len(dMain.columns)):
    dMain.loc[x_][i] = np.nan

In [32]:
#test seed value
seed = dict()
seed['Breeding Pool'] = 4
seed['Misty Rainforest'] = 4
seed['Temple Garden'] = 4



In [33]:
# used to select card fewqeuencies for non lands
def F(x):
    if x < 0:
        return 0
    if x > 3:
        return 4
    else:
        return math.ceil(x)
# generate a 60 card decklist given a seed list of cards/freqs
def genDeck(seed):
    decksize = 0
    imputer = IterativeImputer(BayesianRidge())
    #impute current seed
    for s in seed:
        decksize+=seed[s]
    while decksize!=60:
        for card in dMain:
            if card in seed:
                dMain.loc[x_][card] = seed[card]
            else:
                dMain.loc[x_][card] = np.nan
        #impute deck givne current seed        
        impMain = pd.DataFrame(imputer.fit_transform(dMain.values),columns = dMain.columns)
        for card in impMain:
            if card in ['Forest', 'Mountain', 'Island', 'Plains', 'Swamp', 'Wastes']:
                impMain[card] = impMain[card].apply(lambda x: math.ceil(x))
            else:
                impMain[card] = impMain[card].apply(F)
        #sort cards not in seed by their average correlatoin w the cards in the seed
        corrs = corrMain[list(seed.keys())].transpose().mean().sort_values(ascending=False).to_dict()
        to_inc = []
        i = 10
        #include top 10 or so cards into the seed (by frequency) at a time then reimpute till 60 cards in deck
        for c in corrs:
            newFreq = impMain.loc[x_][c]
            if c not in seed and newFreq >0:
                if decksize+newFreq<=60:
                    seed[c] = newFreq
                    decksize+=newFreq
                    i-=newFreq
                if i<=0:
                    break
    

In [34]:
genDeck(seed)

C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\joey\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [35]:
seed

{'Breeding Pool': 4,
 'Misty Rainforest': 4,
 'Temple Garden': 4,
 'Path to Exile': 4,
 'Ice-Fang Coatl': 4,
 'Force of Negation': 4,
 "Arcum's Astrolabe": 4,
 'Teferi, Time Raveler': 3,
 'Snow-Covered Island': 4,
 'Jace, the Mind Sculptor': 2,
 'Field of Ruin': 2,
 'Hallowed Fountain': 2,
 'Snow-Covered Plains': 1,
 'Flooded Strand': 3,
 'Mana Leak': 3,
 'Cryptic Command': 1,
 'Snow-Covered Forest': 2,
 'Mystic Sanctuary': 3,
 'Supreme Verdict': 2,
 'Snapcaster Mage': 1,
 "Uro, Titan of Nature's Wrath": 3}

In [36]:
Decksize = 60
#For a given partial decklist, check if the addition of a new card
# satisfies the restrictions placed upon the deck by specifying a companion(s) for it
# and return True if the addition of a new card would not violate the deck restrictions
def meetsCRestrictions(Companions: list, Card: pd.Series(), Decklist: list) -> bool:
    for i in Companions:
        if i == "Lurrus of the Dream-Den" and ('Creature' in Card.types.tolist()[0] or 'Enchantment' in Card.types.tolist()[0] or 'Artifact' in Card.types.tolist()[0] or 'Planeswalker' in Card.types.tolist()[0]) and Card.CMC.tolist()[0] > 2:
            return False
        if i == "Keruga, the Macrosage" and 'Land' not in Card.types.tolist()[0] and Card.CMC.tolist()[0] < 3:
            return False
        if i == "Jegantha, the Wellspring" and Card.manaCost.tolist()[0].split('}') != len(set(Card.manaCost.tolist()[0].split('}'))):
            return False
        if i == "Obosh, the Preypiercer" and 'Land' not in Card.types.tolist()[0]  and Card.CMC.tolist()[0] %2 == 0:
            return False
        if i == "Gyruda, Doom of Depths" and 'Land' not in Card.types.tolist()[0]  and Card.CMC.tolist()[0] % 2 == 1:
            return False
        if i == "Umori, the Collector":
            CardType = None
            for ty in Decklist:
                for x in Decklist[ty]:
                    name = ' '.join(x.split()[1:])
                    x = cards[cards['name']==name]
                    if CardType == None:
                        CardType = x.types.tolist()[0]
                    elif 'Land' in CardType:
                        CardType = x.types.tolist()[0]
                    elif len(x.types.tolist()[0])>1 and 'Land' not in x.types.tolist()[0]:
                        return False
                    else:
                        if x.types.tolist()[0] != ["Land"] and x.types.tolist()[0] not in CardType:
                            return False
        if i == "Lutri, the Spellchaser":
            for ty in Decklist:
                for x in Decklist[ty]:
                    freq = int(x.split()[0])
                    name  = ' '.join(x.split()[1:])
                    if (freq >1 and ty!='LANDS') or name==Card.name.tolist()[0]:
                        return False

        if i == "Kaheera, the Orphaguard" and 'Creature' in Card.types.tolist()[0]  and ("Cat" not in Card.subtypes.tolist()[0]  or "Elemental" not in Card.subtypes.tolist()[0] or "Nightmare" not in Card.subtypes.tolist()[0] or "Dinosaur" not in Card.subtypes.tolist()[0] or "Beast" not in Card.subtypes.tolist()[0]):
            return False
        if i == "Yorion, Sky Nomad":
            Decksize = 80
    return True

C:\Users\joey\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [37]:
#Sample decklist to check companion restrictions and recommended cards
sampleDecklist = {}
sampleDecklist["LANDS"] = []
sampleDecklist["CREATURES"] = ["4 Monastery Swiftspear", "2 Magus of the Moon", "4 Goblin Guide"]
sampleDecklist["INSTANTS and SORC."] = ["4 Lightning Bolt", "4 Boros Charm"]
sampleDecklist["SIDEBOARD"] = ["1 Lurrus of the Dream-Den"]
name = 'Lava Axe'
comp = "Kaheera, the Orphaguard"
meetsCRestrictions([comp],cards[cards['name']==name],Decklist=sampleDecklist)


True

In [38]:
import copy
Companion = []
CompanionList = ["Lurrus of the Dream-Den","Keruga, the Macrosage", "Jegantha, the Wellspring", "Obosh, the Preypiercer", "Gyruda, Doom of Depths", 
                 "Umori, the Collector", "Lutri, the Spellchaser", "Kaheera, the Orphaguard", "Yorion, Sky Nomad"]
def GetRecommended(Data: pd.DataFrame(), IncDeck: dict) -> dict:
    recommendationDeck = pd.Series()
    mainDeck = copy.deepcopy(IncDeck)
    CurrSize = 0
    if 'SIDEBOARD' in IncDeck.keys():
        del mainDeck['SIDEBOARD']
        sideDeck = IncDeck['SIDEBOARD']
        for x in sideDeck:
            x = ' '.join(x.split()[1:])
            potentialCard = cards[cards['name']==x]
            if potentialCard.name.tolist()[0] in CompanionList:
                Companion.append(potentialCard.name.tolist()[0])
    for i in mainDeck.values():
        for x in i:
            print(x)
            nameVal = x.split(" ")
            CurrSize += int(nameVal[0])
    print("Current Deck Size: " + str(CurrSize))
    if len(Companion) != 0:
        print("Companion: " + str(Companion))
    while CurrSize < Decksize:
        CurrSize +=1

In [39]:
GetRecommended(corrMain, sampleDecklist)

4 Monastery Swiftspear
2 Magus of the Moon
4 Goblin Guide
4 Lightning Bolt
4 Boros Charm
Current Deck Size: 18
Companion: ['Lurrus of the Dream-Den']


C:\Users\joey\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [40]:
def RunTests(Decklist):
    newlist = Decklist["INSTANTS and SORC."]+ Decklist["CREATURES"] + Decklist["LANDS"] + Decklist["OTHER SPELLS"]
    for i in newlist:
        i.split(" ")
        " ".join(i[1:])
        tuple(i)
    Testing = newlist[0:len(newlist)/2]
    FinalList = Impute(seed = [(x,y) for x,y in Testing])
    FinalList = FinalList["INSTANTS and SORC."]+ FinalList["CREATURES"] + FinalList["LANDS"] + FinalList["OTHER SPELLS"]
    for i in FinalList:
        i.split(" ")
        " ".join(i[1:])
        tuple(i)
    count = 0
    for i in FinalList:
        if i in newlist:
            count +=1
    return count/len(newlist)